## Prueba de OpenScoring

### Setup

Antes hay que instalar y levantar el servicio que está implementado en Java.

```
version=2.1-SNAPSHOT
cd openscoring-server/target
java -jar openscoring-server-executable-${version}.jar --port 8181

```

La librería que usa el cliente se instala con:

```
pip install --upgrade git+https://github.com/sebastianf101/openscoring-python.git
```

In [1]:
import pandas as pd
import time
from openscoring import Openscoring
from openscoring import EvaluationRequest

### Conexión al servicio

In [2]:

os = Openscoring(base_url = "http://localhost:8181/openscoring")

### Carga de modelo básico

Cargamos un árbol de decisión básico. 

In [3]:
os.deployFile("Iris", "openscoring/tests/resources/DecisionTreeIris.pmml")



### Evaluación individual

In [4]:
arguments = {
	"Sepal.Length" : 5.1,
	"Sepal.Width" : 3.5,
	"Petal.Length" : 1.4,
	"Petal.Width" : 0.2
}

results = os.evaluate("Iris", arguments)
print(results)

{'Species': 'setosa', 'probability(setosa)': 1.0, 'probability(versicolor)': 0.0, 'probability(virginica)': 0.0}


In [5]:
evaluationRequest = EvaluationRequest("record-001", arguments)

evaluationResponse = os.evaluate("Iris", evaluationRequest)
print(evaluationResponse.results)

{'Species': 'setosa', 'probability(setosa)': 1.0, 'probability(versicolor)': 0.0, 'probability(virginica)': 0.0}


### Evaluación por Lotes

Evaluamos las filas del csv input con el modelo cargado y ponemos el resultado en el csv output.

In [6]:
os.evaluateCsvFile("Iris", "openscoring/tests/resources/input.csv", "openscoring/tests/resources/output.csv")

In [7]:
df = pd.read_csv("openscoring/tests/resources/output.csv")
print(df)

           Id     Species  probability(setosa)  probability(versicolor)  \
0  record-001      setosa                  1.0                 0.000000   
1  record-002  versicolor                  0.0                 0.907407   
2  record-003   virginica                  0.0                 0.021739   

   probability(virginica)  
0                0.000000  
1                0.092593  
2                0.978261  


### Prueba Modelo con transformaciones

En otro cuaderno (debido al cambio de entorno) intentamos exportar con formato PMML el modelo que se hizo para el Macro con el módulo `sklearn2pmml` que forma parte de la oferta de productos de OpenScoring.   

Recortando el pipeline del modelo dejando sólo los pasos que usan sci-kit learn "puro" se pudo exportar a un archivo PMML pero luego no se pudo realizar el `deploy`.   

Por ejemplo, el siguiente recorte no funcionó.  

```
model_pipe = PMMLPipeline(steps=[
    ('ImputarMedias', SimpleImputer(missing_values=np.nan, strategy='mean')),
    ('Discretizar', KBinsDiscretizer(encode="onehot", random_state=0)),
    ('LogReg', LogisticRegression(random_state=0))
])
```

Si quitamos el paso 'Discretizar' si funciona el `deploy` y la evaluación. 

Luego probamos apartarnos más de lo hecho para el Macro y seguir en cambio el método de construcción del Pipeline que figura en la documentación: https://openscoring.io/blog/2020/01/19/converting_logistic_regression_pmml/#scikit-learn .

Esto requiere otro módulo, `sklearn_pandas`, para realizar el preprocesamiento.   Probamos realizar la Discretización de esta forma. 

```
# Preproceso
from sklearn.preprocessing import KBinsDiscretizer, StandardScaler, OneHotEncoder
from sklearn_pandas import DataFrameMapper
from sklearn2pmml.decoration import CategoricalDomain, ContinuousDomain
from sklearn2pmml.preprocessing import ExpressionTransformer, LookupTransformer
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer

# Modelos
from sklearn.linear_model import LogisticRegression

mapper = DataFrameMapper([
	(["income", "Credit_Score"], [ContinuousDomain(), SimpleImputer(missing_values=np.nan, strategy='mean'), 
        KBinsDiscretizer(encode="onehot", random_state=101)
      ]), 
	(["occupancy_type"], [CategoricalDomain(), OneHotEncoder(drop = "first")])
])

model_pipe = PMMLPipeline([
	("mapper", mapper),
	("classifier", LogisticRegression(random_state=101))
])
```

y esta vez funcionó! Pudimos exportarlo a formato PMML en el archivo `Modelo_logistico_v1_ALL.pmml.xml`.

```
X_train_rec = X_train[["income", "Credit_Score", "occupancy_type"]]
model_pipe.fit(X_train_rec, y_train)
sklearn2pmml(model_pipe, "Modelo_logistico_v1_ALL.pmml.xml")
```

En Conclusión, **no** se puede exportar cualquier Pipeline,  **tiene** que estar construído de cierta manera.   
Dudo que la librería pueda tampoco evaluar cualquier Pipeline en formato PMML. 
Para eliminar esta duda habría que conseguir ejemplos realistas de pipelines en PMML. 

Ahora vamos a verificar el 'deployment' y medir los tiempos de evaluación de un lote. 


In [8]:
os.deployFile("LogReg", "Modelo_logistico_v1_ALL.pmml.xml")


In [9]:
start_time = time.time()

os.evaluateCsvFile("LogReg", "X_train_rec.csv", "X_train_rec_eval.csv")

end_time = time.time()

In [10]:
df = pd.read_csv("X_train_rec_eval.csv")
print(df.sample(10))

       Status  probability(0)  probability(1)
19766       0        0.822634        0.177366
65207       0        0.823671        0.176329
23564       0        0.631101        0.368899
35626       0        0.823671        0.176329
93864       0        0.739689        0.260311
46091       0        0.631101        0.368899
88934       0        0.817338        0.182662
21150       0        0.633957        0.366043
64679       0        0.633957        0.366043
71194       0        0.807500        0.192500


### Evaluación de tiempos

In [11]:
print(f"La evaluación por 1000 filas tardó: {1000*(end_time-start_time)/df.shape[0]} segundos")

La evaluación por 1000 filas tardó: 0.4654071747973433 segundos


Teniendo en cuenta que el pipeline fue muy liviano, de pocos pasos y variables la evaluación no fué tan rápida.   
Habría que probar con casos más realistas. 

### Posibles próximos pasos

1.  Evaluar casos más realistas de PMML.  ¿El Cliente va a incluir transformaciones en el PMML?  ¿Qué tiempos de respuesta se esperan?  ¿Será online, batch o ambos el uso esperado?    

2. Costó mucho el establecimiento del entorno adecuado tanto para el cliente como para el servicio. 
No obstante para producción el autor de la librería recomienda para montar un servicio aislado y con autorización usar las implementaciones Docker ó AWS: 
"The default user authorization logic is implemented by the org.openscoring.service.filters.NetworkSecurityContextFilter JAX-RS filter class, which grants “user” role (read-only) to any address and “admin” role (read and write) to local host addresses.  When looking to upgrade to a more production-like setup, then Openscoring-Docker and Openscoring-Elastic-Beanstalk projects provide good starting points."